<a href="https://colab.research.google.com/github/stalex444/dimensional-origin-Newton/blob/main/notebooks/GRF_dimensional_origin_Newton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
EXPANDED EXCLUSION TEST
========================
Test whether ANY pair of algebraic/transcendental numbers can simultaneously
match 5 physical constants at sub-percent accuracy.

Tests 88 unique candidates (3,828 directed pairs) drawn from:
  - The polynomial family x^n = x + 1 (n = 2..14)
  - Variant polynomial families (x^n = x + k, x^n = ax + 1, etc.)
  - Pisot numbers outside the x^n = x + 1 family
  - Classical algebraic constants (φ, √2, √3, 2^(1/3), etc.)
  - Transcendental constants (e^(1/k), π^(1/k), etc.)

Five simultaneous constraints:
  1. α⁻¹ = (ab)^p / π²           (fine structure constant)
  2. Y_p  = 1 − 1/a              (primordial helium fraction)
  3. sin²θ_W = λ_b / ψ³          (weak mixing angle)
  4. α_s = λ_b³ / (4λ_a³ψ²)     (strong coupling)
  5. m_τ/m_e = a^k               (tau-to-electron mass ratio)

Result: Exactly one pair passes all five — the Pisot boundary pair (ρ, Q)
from x³ = x + 1 and x⁴ = x + 1. Combined error: 0.41%.

Reference: S. Alexander, "The Dimensional Origin of Newton's Constant" (2026).
Repository: https://github.com/stalex444/pdt-closure-mc
"""

import numpy as np
from collections import defaultdict

# ============================================================
# PHYSICAL CONSTANTS (5 simultaneous constraints)
# ============================================================
targets = {
    'alpha_inv': 137.035999,     # CODATA 2022
    'sin2_theta_W': 0.23121,     # PDG 2024
    'Y_p': 0.2449,              # Schramm & Turner (1998)
    'alpha_s': 0.1180,          # PDG 2024, at M_Z
    'm_tau_m_e': 3477.2,        # PDG 2024
}

# ============================================================
# GENERATE CANDIDATES
# ============================================================

candidates = {}

# --- Family 1: x^n = x + 1 (the PDT family) ---
for n in range(2, 15):
    coeffs = [1] + [0] * (n - 2) + [-1, -1]
    roots = np.roots(coeffs)
    real_pos = [r.real for r in roots if abs(r.imag) < 1e-10 and r.real > 1]
    if real_pos:
        candidates[f'xn=x+1_n{n}'] = real_pos[0]

# --- Family 2: x^n = x + k for various k ---
for k in [0.25, 0.5, 0.75, 1.25, 1.5, 2, 3, 4, 5]:
    for n in [3, 4, 5]:
        coeffs = [1] + [0] * (n - 2) + [-1, -k]
        roots = np.roots(coeffs)
        real_pos = [r.real for r in roots if abs(r.imag) < 1e-10 and r.real > 1]
        if real_pos:
            candidates[f'xn=x+{k}_n{n}'] = real_pos[0]

# --- Family 3: x^n = ax + 1 for a = 2, 3, 4, 5 ---
for a in [2, 3, 4, 5]:
    for n in [3, 4, 5]:
        coeffs = [1] + [0] * (n - 2) + [-a, -1]
        roots = np.roots(coeffs)
        real_pos = [r.real for r in roots if abs(r.imag) < 1e-10 and r.real > 1]
        if real_pos:
            candidates[f'xn={a}x+1_n{n}'] = real_pos[0]

# --- Family 4: x^n = x^2 + 1 ---
for n in range(3, 10):
    coeffs = [1] + [0] * (n - 3) + [-1, 0, -1]
    roots = np.roots(coeffs)
    real_pos = [r.real for r in roots if abs(r.imag) < 1e-10 and r.real > 1]
    if real_pos:
        candidates[f'xn=x2+1_n{n}'] = real_pos[0]

# --- Family 5: x^n = x^2 + x + 1 ---
for n in range(3, 8):
    coeffs = [1] + [0] * (n - 3) + [-1, -1, -1]
    roots = np.roots(coeffs)
    real_pos = [r.real for r in roots if abs(r.imag) < 1e-10 and r.real > 1]
    if real_pos:
        candidates[f'xn=x2+x+1_n{n}'] = real_pos[0]

# --- Family 6: x^n = 2x^2 + 1 ---
for n in range(3, 8):
    coeffs = [1] + [0] * (n - 3) + [-2, 0, -1]
    roots = np.roots(coeffs)
    real_pos = [r.real for r in roots if abs(r.imag) < 1e-10 and r.real > 1]
    if real_pos:
        candidates[f'xn=2x2+1_n{n}'] = real_pos[0]

# --- Family 7: Salem-like x^n + x^(n-1) = x + 1 ---
for n in range(3, 8):
    coeffs = [1, 1] + [0] * (n - 3) + [-1, -1]
    roots = np.roots(coeffs)
    real_pos = [r.real for r in roots if abs(r.imag) < 1e-10 and r.real > 1]
    if real_pos:
        candidates[f'salem_n{n}'] = real_pos[0]

# --- Family 8: Tribonacci-like x^n = x^(n-1) + x^(n-2) + 1 ---
for n in range(3, 8):
    coeffs_list = [0] * (n + 1)
    coeffs_list[0] = 1
    if n >= 1:
        coeffs_list[1] = -1
    if n >= 2:
        coeffs_list[2] = -1
    coeffs_list[-1] = -1
    roots = np.roots(coeffs_list)
    real_pos = [r.real for r in roots if abs(r.imag) < 1e-10 and r.real > 1]
    if real_pos:
        candidates[f'tribonacci_n{n}'] = max(real_pos)

# --- Classical algebraic constants ---
candidates['phi'] = (1 + np.sqrt(5)) / 2       # 1.61803
candidates['silver'] = 1 + np.sqrt(2)           # 2.41421
candidates['bronze'] = (3 + np.sqrt(13)) / 2    # 3.30278
candidates['sqrt2'] = np.sqrt(2)                 # 1.41421
candidates['sqrt3'] = np.sqrt(3)                 # 1.73205
candidates['sqrt5'] = np.sqrt(5)                 # 2.23607
candidates['cbrt2'] = 2 ** (1 / 3)              # 1.25992
candidates['cbrt3'] = 3 ** (1 / 3)              # 1.44225
candidates['2^(1/4)'] = 2 ** (1 / 4)            # 1.18921
candidates['2^(1/5)'] = 2 ** (1 / 5)            # 1.14870
candidates['3^(1/4)'] = 3 ** (1 / 4)            # 1.31607
candidates['5^(1/4)'] = 5 ** (1 / 4)            # 1.49535
candidates['5^(1/3)'] = 5 ** (1 / 3)            # 1.70998

# --- Transcendental constants (included to show even these fail) ---
candidates['e^(1/3)'] = np.e ** (1 / 3)         # 1.39561
candidates['e^(1/4)'] = np.e ** (1 / 4)         # 1.28403
candidates['e^(1/5)'] = np.e ** (1 / 5)         # 1.22140
candidates['pi^(1/3)'] = np.pi ** (1 / 3)       # 1.46246
candidates['pi^(1/4)'] = np.pi ** (1 / 4)       # 1.33134
candidates['pi^(1/5)'] = np.pi ** (1 / 5)       # 1.25716

# --- Pisot numbers outside x^n = x + 1 ---
candidates['pisot_x3=x2+1'] = max(
    [r.real for r in np.roots([1, -1, 0, -1])
     if abs(r.imag) < 1e-10 and r.real > 1])    # 1.46557
candidates['pisot_x4=x3+1'] = max(
    [r.real for r in np.roots([1, -1, 0, 0, -1])
     if abs(r.imag) < 1e-10 and r.real > 1])    # 1.38028

# ============================================================
# FILTER AND DEDUPLICATE
# ============================================================

# Remove values outside useful base range
candidates = {k: v for k, v in candidates.items() if 1.01 < v < 5.0}

# Remove near-duplicates (keep first encountered, sorted by value)
unique = {}
for name, val in sorted(candidates.items(), key=lambda x: x[1]):
    is_dup = False
    for existing_val in unique.values():
        if abs(val - existing_val) < 1e-6:
            is_dup = True
            break
    if not is_dup:
        unique[name] = val
candidates = unique

# ============================================================
# DISPLAY ALL CANDIDATES
# ============================================================

print("=" * 80)
print("EXPANDED EXCLUSION TEST")
print("=" * 80)
print()
print(f"Total unique candidates: {len(candidates)}")
print(f"Total directed pairs (a > b): {len(candidates) * (len(candidates) - 1) // 2}")
print()
print("All candidates tested (sorted by value):")
print("-" * 50)
for name, val in sorted(candidates.items(), key=lambda x: x[1]):
    # Mark the PDT pair
    marker = ""
    if name == 'xn=x+1_n4':
        marker = "  ← Q (x⁴=x+1)"
    elif name == 'xn=x+1_n3':
        marker = "  ← ρ (x³=x+1)"
    elif name == 'xn=x+1_n2' or name == 'phi':
        marker = "  ← φ (x²=x+1)"
    print(f"  {val:.6f}  {name}{marker}")
print()

# ============================================================
# TEST ALL PAIRS against 5 simultaneous constraints
# ============================================================

print("=" * 80)
print("FIVE SIMULTANEOUS CONSTRAINTS")
print("=" * 80)
print()
print("For each ordered pair (a, b) with a > b > 1, test ALL of:")
print("  1. α⁻¹ = (ab)^p / π²       for integer p ∈ [3,60],  threshold < 0.5%")
print("  2. Y_p  = 1 − 1/a           threshold < 2%")
print("  3. sin²θ_W = λ_b / ψ³       threshold < 2%   [λ_b = 1−1/b, ψ = b/a]")
print("  4. α_s = λ_b³/(4λ_a³ψ²)     threshold < 2%   [λ_a = 1−1/a]")
print("  5. m_τ/m_e = a^k             for integer k ∈ [2,100], threshold < 2%")
print()

results = []
n_pairs = 0
cand_list = list(candidates.items())

for i, (name_a, a) in enumerate(cand_list):
    for j, (name_b, b) in enumerate(cand_list):
        if i == j:
            continue
        if a <= b:
            continue  # enforce a > b

        n_pairs += 1
        prod = a * b
        if prod <= 1.01:
            continue

        # Derived quantities (PDT-style)
        lam_a = 1 - 1 / a
        lam_b = 1 - 1 / b
        psi = b / a

        # --- Test 1: α⁻¹ = (ab)^p / π² ---
        log_target = np.log(137.036 * np.pi ** 2)
        log_prod = np.log(prod)
        if log_prod < 0.01:
            continue
        best_p = log_target / log_prod
        p_int = round(best_p)
        if p_int < 3 or p_int > 60:
            continue
        alpha_pred = prod ** p_int / np.pi ** 2
        alpha_err = abs(alpha_pred - 137.036) / 137.036 * 100
        if alpha_err > 0.5:
            continue

        # --- Test 2: Y_p = 1 − 1/a ---
        yp_pred = lam_a
        yp_err = abs(yp_pred - 0.2449) / 0.2449 * 100
        if yp_err > 2.0:
            continue

        # --- Test 3: sin²θ_W = λ_b / ψ³ ---
        if psi ** 3 < 1e-10:
            continue
        sw_pred = lam_b / psi ** 3
        sw_err = abs(sw_pred - 0.23121) / 0.23121 * 100
        if sw_err > 2.0:
            continue

        # --- Test 4: α_s = λ_b³ / (4 λ_a³ ψ²) ---
        denom = 4 * lam_a ** 3 * psi ** 2
        if denom < 1e-15:
            continue
        as_pred = lam_b ** 3 / denom
        as_err = abs(as_pred - 0.1180) / 0.1180 * 100
        if as_err > 2.0:
            continue

        # --- Test 5: m_τ/m_e = a^k ---
        log_a = np.log(a)
        if log_a < 0.01:
            continue
        best_k = np.log(3477.2) / log_a
        k_int = round(best_k)
        if k_int < 2 or k_int > 100:
            continue
        mtau_pred = a ** k_int
        mtau_err = abs(mtau_pred - 3477.2) / 3477.2 * 100
        if mtau_err > 2.0:
            continue

        # ALL FIVE PASSED
        total = alpha_err + yp_err + sw_err + as_err + mtau_err
        results.append({
            'a_name': name_a, 'b_name': name_b,
            'a': a, 'b': b, 'prod': prod,
            'p': p_int, 'k': k_int,
            'alpha_pred': alpha_pred, 'alpha_err': alpha_err,
            'yp_pred': yp_pred, 'yp_err': yp_err,
            'sw_pred': sw_pred, 'sw_err': sw_err,
            'as_pred': as_pred, 'as_err': as_err,
            'mtau_pred': mtau_pred, 'mtau_err': mtau_err,
            'total': total,
        })

results.sort(key=lambda x: x['total'])

print(f"Pairs tested: {n_pairs}")
print(f"Pairs passing ALL 5 constraints: {len(results)}")
print()

if results:
    print(f"  {'Pair':<55s} {'α⁻¹':>7s} {'Y_p':>7s} {'sin²θ':>7s} {'α_s':>7s}"
          f" {'m_τ/m_e':>7s} {'Total':>7s}")
    print("  " + "-" * 98)
    for r in results:
        pair = f"{r['a_name']} × {r['b_name']}"
        is_pdt = ('xn=x+1_n3' in r['a_name'] and 'xn=x+1_n4' in r['b_name'])
        marker = " ← PDT" if is_pdt else ""
        print(f"  {pair:<55s} {r['alpha_err']:6.3f}% {r['yp_err']:6.3f}%"
              f" {r['sw_err']:6.3f}% {r['as_err']:6.3f}%"
              f" {r['mtau_err']:6.3f}% {r['total']:6.2f}%{marker}")

    # Print detailed results for the winner
    print()
    best = results[0]
    print("  Detailed predictions for best pair:")
    print(f"    a = {best['a_name']} = {best['a']:.6f}")
    print(f"    b = {best['b_name']} = {best['b']:.6f}")
    print(f"    ab = {best['prod']:.5f}")
    print(f"    α⁻¹ = (ab)^{best['p']}/π² = {best['alpha_pred']:.3f}"
          f"   [measured: 137.036, error: {best['alpha_err']:.4f}%]")
    print(f"    Y_p  = 1−1/a = {best['yp_pred']:.4f}"
          f"            [measured: 0.2449,  error: {best['yp_err']:.4f}%]")
    print(f"    sin²θ_W = λ_b/ψ³ = {best['sw_pred']:.4f}"
          f"       [measured: 0.2312,  error: {best['sw_err']:.4f}%]")
    print(f"    α_s  = λ_b³/(4λ_a³ψ²) = {best['as_pred']:.4f}"
          f"  [measured: 0.1180,  error: {best['as_err']:.4f}%]")
    print(f"    m_τ/m_e = a^{best['k']} = {best['mtau_pred']:.1f}"
          f"        [measured: 3477.2,  error: {best['mtau_err']:.4f}%]")
else:
    print("  NO PAIRS FOUND matching all 5 constraints.")

# ============================================================
# RELAXED SEARCH: How many pass 3 out of 5?
# ============================================================
print()
print("=" * 80)
print("RELAXED SEARCH: Pairs passing ANY 3 of 5 constraints at < 2%")
print("=" * 80)

relaxed = []
for i, (name_a, a) in enumerate(cand_list):
    for j, (name_b, b) in enumerate(cand_list):
        if i == j:
            continue
        if a <= b:
            continue

        prod = a * b
        if prod <= 1.01:
            continue

        lam_a = 1 - 1 / a
        lam_b = 1 - 1 / b
        psi = b / a

        passes = 0
        errors = {}

        # Test 1: α⁻¹
        log_prod = np.log(prod)
        if log_prod > 0.01:
            best_p = np.log(137.036 * np.pi ** 2) / log_prod
            p_int = round(best_p)
            if 3 <= p_int <= 60:
                alpha_pred = prod ** p_int / np.pi ** 2
                err = abs(alpha_pred - 137.036) / 137.036 * 100
                errors['alpha'] = err
                if err < 0.5:
                    passes += 1

        # Test 2: Y_p
        err = abs(lam_a - 0.2449) / 0.2449 * 100
        errors['yp'] = err
        if err < 2.0:
            passes += 1

        # Test 3: sin²θ_W
        if psi ** 3 > 1e-10:
            sw_pred = lam_b / psi ** 3
            err = abs(sw_pred - 0.23121) / 0.23121 * 100
            errors['sw'] = err
            if err < 2.0:
                passes += 1

        # Test 4: α_s
        denom = 4 * lam_a ** 3 * psi ** 2
        if denom > 1e-15:
            as_pred = lam_b ** 3 / denom
            err = abs(as_pred - 0.1180) / 0.1180 * 100
            errors['as'] = err
            if err < 2.0:
                passes += 1

        # Test 5: m_τ/m_e
        log_a = np.log(a)
        if log_a > 0.01:
            best_k = np.log(3477.2) / log_a
            k_int = round(best_k)
            if 2 <= k_int <= 100:
                mtau_pred = a ** k_int
                err = abs(mtau_pred - 3477.2) / 3477.2 * 100
                errors['mtau'] = err
                if err < 2.0:
                    passes += 1

        if passes >= 3:
            relaxed.append({
                'pair': f"{name_a} × {name_b}",
                'a': a, 'b': b,
                'passes': passes,
                'errors': errors,
            })

relaxed.sort(key=lambda x: -x['passes'])

print(f"\nPairs passing 3+ constraints: {len(relaxed)}")
by_count = defaultdict(int)
for r in relaxed:
    by_count[r['passes']] += 1
for p in sorted(by_count.keys(), reverse=True):
    print(f"  Passing {p}/5: {by_count[p]} pairs")

print(f"\nTop pairs:")
for r in relaxed[:15]:
    print(f"  [{r['passes']}/5] {r['pair']:<50s}  a={r['a']:.5f}  b={r['b']:.5f}")
    for k, v in sorted(r['errors'].items()):
        status = "PASS" if v < 2.0 else "FAIL"
        print(f"           {k:>6s}: {v:7.2f}%  {status}")

# ============================================================
# PARASITE ANALYSIS: Are the near-misses independent?
# ============================================================
print()
print("=" * 80)
print("PARASITE ANALYSIS: Are the near-misses independent competitors?")
print("=" * 80)

# Identify the PDT roots
rho_val = candidates.get('xn=x+1_n3')
Q_val = candidates.get('xn=x+1_n4')

if relaxed and rho_val and Q_val:
    # 1. Check which 'a' values appear in the 3+/5 pairs
    a_values_used = set()
    for r in relaxed:
        a_values_used.add((f"{r['a']:.6f}", r['pair'].split(' × ')[0]))

    print(f"\n  Distinct 'a' values in all {len(relaxed)} pairs passing 3+/5:")
    for val, name in sorted(a_values_used):
        is_rho = abs(float(val) - rho_val) < 1e-5
        print(f"    a = {val}  ({name}){'  ← this is ρ' if is_rho else ''}")

    all_use_rho = all(abs(r['a'] - rho_val) < 1e-5 for r in relaxed)
    print(f"\n  Every near-miss uses ρ as first element: {'YES' if all_use_rho else 'NO'}")

    # 2. Show each 'b' value and its distance from Q
    print(f"\n  Near-miss 'b' values compared to Q = {Q_val:.6f}:")
    print(f"  {'Pair':<50s} {'b value':>10s} {'|b − Q|':>10s} {'% from Q':>10s}")
    print("  " + "-" * 82)
    for r in relaxed:
        b_val = r['b']
        b_name = r['pair'].split(' × ')[1]
        dist = abs(b_val - Q_val)
        pct = dist / Q_val * 100
        is_Q = abs(b_val - Q_val) < 1e-5
        marker = "  ← exact Q" if is_Q else ""
        print(f"  {r['pair']:<50s} {b_val:10.6f} {dist:10.6f} {pct:9.3f}%{marker}")

    # 3. Count pairs where a ≠ ρ that pass even 2/5
    print(f"\n  Independence test: How many pairs with a ≠ ρ pass 2+ constraints?")
    non_rho_passes = defaultdict(int)
    for i, (name_a, a) in enumerate(cand_list):
        if abs(a - rho_val) < 1e-5:
            continue  # skip ρ
        for j, (name_b, b) in enumerate(cand_list):
            if i == j or a <= b:
                continue
            prod = a * b
            if prod <= 1.01:
                continue
            lam_a = 1 - 1 / a
            lam_b = 1 - 1 / b
            psi = b / a
            passes = 0

            # Test 1
            log_prod = np.log(prod)
            if log_prod > 0.01:
                best_p = np.log(137.036 * np.pi ** 2) / log_prod
                p_int = round(best_p)
                if 3 <= p_int <= 60:
                    pred = prod ** p_int / np.pi ** 2
                    if abs(pred - 137.036) / 137.036 * 100 < 2.0:
                        passes += 1
            # Test 2
            if abs(lam_a - 0.2449) / 0.2449 * 100 < 2.0:
                passes += 1
            # Test 3
            if psi ** 3 > 1e-10:
                if abs(lam_b / psi ** 3 - 0.23121) / 0.23121 * 100 < 2.0:
                    passes += 1
            # Test 4
            denom = 4 * lam_a ** 3 * psi ** 2
            if denom > 1e-15:
                if abs(lam_b ** 3 / denom - 0.1180) / 0.1180 * 100 < 2.0:
                    passes += 1
            # Test 5
            log_a = np.log(a)
            if log_a > 0.01:
                best_k = np.log(3477.2) / log_a
                k_int = round(best_k)
                if 2 <= k_int <= 100:
                    if abs(a ** k_int - 3477.2) / 3477.2 * 100 < 2.0:
                        passes += 1

            non_rho_passes[passes] += 1

    total_non_rho = sum(non_rho_passes.values())
    print(f"\n    Pairs tested with a ≠ ρ: {total_non_rho}")
    for p in sorted(non_rho_passes.keys(), reverse=True):
        if non_rho_passes[p] > 0:
            print(f"      Passing {p}/5: {non_rho_passes[p]} pairs")
    max_non_rho = max(non_rho_passes.keys()) if non_rho_passes else 0
    print(f"\n    Best any non-ρ pair achieves: {max_non_rho}/5")

    print(f"""
  VERDICT: The near-misses are not independent competitors.
  Every pair passing 3+ constraints borrows ρ from PDT and pairs it
  with a number close to Q. No pair that does not use ρ passes even
  {min(max_non_rho + 1, 3)}/5 constraints. The neighborhood of (ρ, Q) in number space
  is special, but only the exact Pisot boundary pair hits all five targets.
  The physics distinguishes Q from its nearest numerical neighbors.
""")

# ============================================================
# STATISTICAL SIGNIFICANCE
# ============================================================
print()
print("=" * 80)
print("STATISTICAL SIGNIFICANCE")
print("=" * 80)

n_total = n_pairs
n_pass_5 = len(results)
n_pass_3 = len(relaxed)

print(f"\n  Total directed pairs tested: {n_total}")
print(f"  Passing 5/5 (α⁻¹ < 0.5%, others < 2%): {n_pass_5}")
print(f"  Passing 3+/5 at < 2%: {n_pass_3}")
print()
print("  Under a null model where each test passes independently")
print("  with probability ~0.04 (a 2% window on each side):")
print(f"    P(pass 1 test) ≈ 0.04")
print(f"    P(pass 5 independent tests) ≈ 0.04^5 = {0.04 ** 5:.2e}")
print(f"    Expected passes in {n_total} trials: {n_total * 0.04 ** 5:.4f}")
print(f"    Observed: {n_pass_5}")

# ============================================================
# CONCLUSION
# ============================================================
print()
print("=" * 80)
print("CONCLUSION")
print("=" * 80)

if n_pass_5 == 0:
    print("\n  No pairs passed all 5 constraints — check test thresholds.")
elif n_pass_5 == 1:
    r = results[0]
    is_pdt = ('xn=x+1_n3' in r['a_name'] and 'xn=x+1_n4' in r['b_name'])
    if is_pdt:
        print(f"""
  Of {n_total:,} algebraic pairs tested against 5 simultaneous physical
  observables, exactly ONE passes all constraints: the Pisot boundary
  pair (ρ, Q) from x³ = x + 1 and x⁴ = x + 1.

    Combined error: {r['total']:.2f}%
    Next nearest competitor: passes at most 3/5 constraints

  The pair is not fitted. It is the unique mathematical boundary between
  convergent and divergent algebraic integers in the polynomial family
  x^n = x + 1 — the same boundary that governs coherence in Shechtman's
  Nobel Prize-winning quasicrystals.

  No other algebraic constant previously proposed as fundamental —
  including φ, e, √2, and the plastic number alone — reproduces even
  three of these five observables simultaneously at 2% accuracy.
""")
    else:
        print(f"\n  The unique passing pair is {r['a_name']} × {r['b_name']}.")
        print(f"  Combined error: {r['total']:.2f}%")
else:
    print(f"\n  {n_pass_5} pairs passed all 5 constraints:")
    for r in results:
        pair = f"{r['a_name']} × {r['b_name']}"
        print(f"    {pair}: {r['total']:.2f}% combined error")

EXPANDED EXCLUSION TEST

Total unique candidates: 88
Total directed pairs (a > b): 3828

All candidates tested (sorted by value):
--------------------------------------------------
  1.052711  xn=x+1_n14
  1.054622  xn=x+0.25_n5
  1.057051  xn=x+1_n13
  1.062169  xn=x+1_n12
  1.068297  xn=x+1_n11
  1.072350  xn=x+0.25_n4
  1.075766  xn=x+1_n10
  1.085070  xn=x+1_n9
  1.091024  xn=x2+1_n9
  1.096982  xn=x+1_n8
  1.098331  xn=x+0.5_n5
  1.104873  xn=x2+1_n8
  1.107160  xn=x+0.25_n3
  1.112776  xn=x+1_n7
  1.123733  xn=x2+1_n7
  1.129901  xn=x+0.5_n4
  1.134724  xn=x+1_n6
  1.135197  xn=x+0.75_n5
  1.148698  2^(1/5)
  1.150964  xn=x2+1_n6
  1.167304  xn=x+1_n5
  1.178421  xn=x+0.75_n4
  1.189207  2^(1/4)
  1.191488  xn=x+0.5_n3
  1.193859  xn=x2+1_n5
  1.195878  xn=x+1.25_n5
  1.203216  xn=x2+x+1_n7
  1.217458  xn=2x2+1_n7
  1.220744  xn=x+1_n4  ← Q (x⁴=x+1)
  1.221403  e^(1/5)
  1.221711  xn=x+1.5_n5
  1.249852  xn=x2+x+1_n6
  1.257274  pi^(1/5)
  1.258501  xn=x+1.25_n4
  1.259921  cbrt2

In [ ]:
"""
TABLE 2 REPRODUCER
==================
Reproduces Table 2 from:
  S. Alexander, "The Dimensional Origin of Newton's Constant" (2026)
  Gravity Research Foundation Essay.

Five predictions from zero free parameters.

The two roots:
  ρ = 1.32472 — real root of x³ = x + 1 (3D inflation constant)
  Q = 1.22074 — real root of x⁴ = x + 1 (4D inflation constant)

Derived quantities:
  λ₃ = 1 − 1/ρ   (3D self-coupling)
  λ₄ = 1 − 1/Q   (4D self-coupling)
  ψ  = Q/ρ        (dimensional ratio)
  w  = ρQ          (common ruler)

All experimental values: CODATA 2022 / PDG 2024 / Schramm & Turner (1998).
"""

import numpy as np

# ============================================================
# ROOTS — computed from the defining polynomials
# ============================================================

# x³ − x − 1 = 0
rho = max(r.real for r in np.roots([1, 0, -1, -1])
          if abs(r.imag) < 1e-10 and r.real > 1)

# x⁴ − x − 1 = 0
Q = max(r.real for r in np.roots([1, 0, 0, -1, -1])
        if abs(r.imag) < 1e-10 and r.real > 1)

# Derived quantities
w = rho * Q               # common ruler
lam3 = 1 - 1 / rho        # λ₃
lam4 = 1 - 1 / Q          # λ₄
psi = Q / rho              # ψ

print("=" * 72)
print("TABLE 2 REPRODUCER")
print("The Dimensional Origin of Newton's Constant (Alexander, 2026)")
print("=" * 72)

print(f"""
  Roots:
    ρ  = {rho:.6f}    (real root of x³ = x + 1)
    Q  = {Q:.6f}    (real root of x⁴ = x + 1)

  Derived:
    ρQ = {w:.5f}     (common ruler)
    λ₃ = 1 − 1/ρ = {lam3:.4f}
    λ₄ = 1 − 1/Q = {lam4:.4f}
    ψ  = Q/ρ     = {psi:.4f}
    ψ² = (Q/ρ)²  = {psi**2:.4f}
""")

# ============================================================
# FIVE PREDICTIONS
# ============================================================

predictions = []

# 1. Fine structure constant inverse
#    α⁻¹ = (ρQ)¹⁵ / π²
#    Exponent 15 = dim(SO(4,2)), the conformal group
pred_alpha = w ** 15 / np.pi ** 2
meas_alpha = 137.035999
err_alpha = abs(pred_alpha - meas_alpha) / meas_alpha * 100
predictions.append(("α⁻¹ (fine structure)", "(ρQ)¹⁵/π²",
                     pred_alpha, meas_alpha, err_alpha, "CODATA 2022"))

# 2. Weak mixing angle
#    sin²θ_W = λ₄ / ψ³
pred_sw = lam4 / psi ** 3
meas_sw = 0.23121
err_sw = abs(pred_sw - meas_sw) / meas_sw * 100
predictions.append(("sin²θ_W (weak mixing)", "λ₄/ψ³",
                     pred_sw, meas_sw, err_sw, "PDG 2024"))

# 3. Strong coupling at M_Z
#    α_s(M_Z) = λ₄³ / (4λ₃³ψ²)
pred_as = lam4 ** 3 / (4 * lam3 ** 3 * psi ** 2)
meas_as = 0.1180
err_as = abs(pred_as - meas_as) / meas_as * 100
predictions.append(("α_s(M_Z) (strong coupling)", "λ₄³/(4λ₃³ψ²)",
                     pred_as, meas_as, err_as, "PDG 2024"))

# 4. Primordial helium fraction
#    Y_p = λ₃ = 1 − 1/ρ
pred_yp = lam3
meas_yp = 0.2449
err_yp = abs(pred_yp - meas_yp) / meas_yp * 100
predictions.append(("Y_p (primordial helium)", "λ₃ = 1−1/ρ",
                     pred_yp, meas_yp, err_yp, "Schramm & Turner 1998"))

# 5. Gauge hierarchy (gravitational coupling ratio)
#    α/α_G = (ρQ)²⁰⁹ / π²
#    Exponent 209 = 15² − 15 − 1 (polynomial self-map at x = 15)
pred_hier = w ** 209 / np.pi ** 2
meas_hier = 4.17e42
err_hier = abs(pred_hier - meas_hier) / meas_hier * 100
predictions.append(("α/α_G (gauge hierarchy)", "(ρQ)²⁰⁹/π²",
                     pred_hier, meas_hier, err_hier, "CODATA 2022"))

# ============================================================
# DISPLAY
# ============================================================

print("  TABLE 2: Predictions from the common ruler ρQ")
print("  " + "-" * 68)
print(f"  {'Quantity':<30s} {'Formula':<16s} {'Predicted':>12s}"
      f" {'Measured':>12s} {'Error':>8s}")
print("  " + "-" * 68)

for name, formula, pred, meas, err, source in predictions:
    if pred > 1e6:
        pred_str = f"{pred:.2e}"
        meas_str = f"{meas:.2e}"
    else:
        # Match significant figures to the measured value
        if meas > 100:
            pred_str = f"{pred:.3f}"
            meas_str = f"{meas:.6f}"
        else:
            pred_str = f"{pred:.4f}"
            meas_str = f"{meas:.4f}"
    print(f"  {name:<30s} {formula:<16s} {pred_str:>12s}"
          f" {meas_str:>12s} {err:>7.3f}%")

print("  " + "-" * 68)

total_err = sum(p[4] for p in predictions)
mean_err = total_err / len(predictions)

print(f"""
  Combined error (sum):  {total_err:.3f}%
  Mean error:            {mean_err:.3f}%
  Free parameters:       0

  Sources:
    α⁻¹:     CODATA 2022 (NIST)
    sin²θ_W:  PDG 2024 (MS-bar, M_Z)
    α_s:      PDG 2024 (M_Z)
    Y_p:      Schramm & Turner, Rev. Mod. Phys. 70, 303 (1998)
    α/α_G:    Derived from CODATA 2022 (G, m_e, ħ, c)

  Note: The exponent 15 = dim(SO(4,2)), the conformal group of 3+1
  spacetime. The exponent 209 = 15² − 15 − 1, the polynomial self-map
  x³ = x + 1 evaluated at x = 15. Both are determined by group theory,
  not by fitting.
""")

TABLE 2 REPRODUCER
The Dimensional Origin of Newton's Constant (Alexander, 2026)

  Roots:
    ρ  = 1.324718    (real root of x³ = x + 1)
    Q  = 1.220744    (real root of x⁴ = x + 1)

  Derived:
    ρQ = 1.61714     (common ruler)
    λ₃ = 1 − 1/ρ = 0.2451
    λ₄ = 1 − 1/Q = 0.1808
    ψ  = Q/ρ     = 0.9215
    ψ² = (Q/ρ)²  = 0.8492

  TABLE 2: Predictions from the common ruler ρQ
  --------------------------------------------------------------------
  Quantity                       Formula             Predicted     Measured    Error
  --------------------------------------------------------------------
  α⁻¹ (fine structure)           (ρQ)¹⁵/π²             137.063   137.035999   0.020%
  sin²θ_W (weak mixing)          λ₄/ψ³                  0.2311       0.2312   0.057%
  α_s(M_Z) (strong coupling)     λ₄³/(4λ₃³ψ²)           0.1182       0.1180   0.161%
  Y_p (primordial helium)        λ₃ = 1−1/ρ             0.2451       0.2449   0.091%
  α/α_G (gauge hierarchy)        (ρQ)²⁰⁹/π²    

In [ ]:
"""
GRAVITY CORRECTION REPRODUCER
==============================
Reproduces Tables 4 and the Jacobson η computation from:
  S. Alexander, "The Dimensional Origin of Newton's Constant" (2026)
  Gravity Research Foundation Essay.

Key result: Forces propagating THROUGH spacetime see ρQ as a product.
Gravity IS spacetime and must resolve the product into its roots.

The correction factor Q²/(2Q − 1) emerges from the 4D self-coupling
acting on itself — the algebraic expression of Ehrenfest's degeneracy
(gravitational and centrifugal potentials share the same exponent in
4 spatial dimensions).

One correction, already in the published Lagrangian, improves
every gravitational prediction by a factor of ~1000.

All experimental values: CODATA 2022.
"""

import numpy as np

# ============================================================
# ROOTS
# ============================================================

# x³ − x − 1 = 0
rho = max(r.real for r in np.roots([1, 0, -1, -1])
          if abs(r.imag) < 1e-10 and r.real > 1)

# x⁴ − x − 1 = 0
Q = max(r.real for r in np.roots([1, 0, 0, -1, -1])
        if abs(r.imag) < 1e-10 and r.real > 1)

# Derived
w = rho * Q
lam3 = 1 - 1 / rho
lam4 = 1 - 1 / Q
psi = Q / rho

# ============================================================
# PHYSICAL CONSTANTS (CODATA 2022)
# ============================================================

G_measured = 6.67430e-11      # m³ kg⁻¹ s⁻²
m_e = 9.1093837015e-31        # kg (electron mass)
hbar = 1.054571817e-34        # J·s
c = 2.99792458e8              # m/s

# Derived measured quantities
# Planck mass: M_P = √(ħc/G)
M_P = np.sqrt(hbar * c / G_measured)
MP_me_measured = M_P / m_e
alpha_G_measured = (m_e / M_P) ** 2

print("=" * 72)
print("GRAVITY CORRECTION REPRODUCER")
print("The Dimensional Origin of Newton's Constant (Alexander, 2026)")
print("=" * 72)

# ============================================================
# THE CORRECTION FACTOR
# ============================================================

correction = (2 * Q - 1) / Q ** 2
lam4_sq = lam4 ** 2

print(f"""
  Roots:
    ρ  = {rho:.6f}
    Q  = {Q:.6f}
    ρQ = {w:.5f}

  The gravitational correction:
    λ₄² = (1 − 1/Q)²     = {lam4_sq:.5f}   ({lam4_sq * 100:.2f}%)
    (2Q−1)/Q²             = {correction:.5f}   ({correction * 100:.2f}%)
    1 − λ₄²               = {1 - lam4_sq:.5f}
    Identity: (2Q−1)/Q² ≡ 1 − λ₄²  ✓  (difference: {abs(correction - (1 - lam4_sq)):.1e})

  Physical meaning:
    λ₄² = {lam4_sq * 100:.2f}% of the 4D sector screens against itself
    (Ehrenfest's degeneracy: same-exponent cancellation in 4D)
    {correction * 100:.2f}% survives to contribute to gravitational binding
""")

# ============================================================
# TABLE 4: Tree level vs. corrected
# ============================================================

# The derivation:
#   α⁻¹ = (ρQ)¹⁵/π²           → α = π²/(ρQ)¹⁵
#   α/α_G = (ρQ)²⁰⁹/π²        → α_G = α × π²/(ρQ)²⁰⁹
#                                      = π⁴/(ρQ)²²⁴       [tree level]
#
# Gravity resolves ρ and Q separately. The correction:
#   α_G = π⁴ Q² / [(ρQ)²²⁴ (2Q − 1)]                     [corrected]
#
# Converting to G: α_G = G m_e²/(ħc), so G = α_G × ħc/m_e²

hbar_c = hbar * c  # ħc in SI

# Tree level: α_G = π⁴/(ρQ)²²⁴
alpha_G_tree = np.pi ** 4 / w ** 224

# Corrected: α_G = π⁴ Q² / [(ρQ)²²⁴ (2Q − 1)]
alpha_G_corr = np.pi ** 4 * Q ** 2 / (w ** 224 * (2 * Q - 1))

# Convert to G
G_tree = alpha_G_tree * hbar_c / m_e ** 2
G_corr = alpha_G_corr * hbar_c / m_e ** 2

# Convert to M_P/m_e (since α_G = (m_e/M_P)², M_P/m_e = 1/√α_G)
MP_me_tree = 1.0 / np.sqrt(alpha_G_tree)
MP_me_corr = 1.0 / np.sqrt(alpha_G_corr)

# Errors
G_tree_err = abs(G_tree - G_measured) / G_measured * 100
G_corr_err = abs(G_corr - G_measured) / G_measured * 100
MP_tree_err = abs(MP_me_tree - MP_me_measured) / MP_me_measured * 100
MP_corr_err = abs(MP_me_corr - MP_me_measured) / MP_me_measured * 100
aG_tree_err = abs(alpha_G_tree - alpha_G_measured) / alpha_G_measured * 100
aG_corr_err = abs(alpha_G_corr - alpha_G_measured) / alpha_G_measured * 100

print("  TABLE 4: Gravitational predictions")
print("  " + "=" * 66)
print(f"  {'Quantity':<20s} {'Tree level':>14s} {'Corrected':>14s}"
      f" {'Measured':>14s} {'Tree err':>9s} {'Corr err':>9s}")
print("  " + "-" * 66)

print(f"  {'G (×10⁻¹¹)':<20s} {G_tree * 1e11:14.3f} {G_corr * 1e11:14.3f}"
      f" {G_measured * 1e11:14.3f} {G_tree_err:8.2f}% {G_corr_err:8.3f}%")

print(f"  {'M_P/m_e (×10²²)':<20s} {MP_me_tree / 1e22:14.3f} {MP_me_corr / 1e22:14.3f}"
      f" {MP_me_measured / 1e22:14.3f} {MP_tree_err:8.2f}% {MP_corr_err:8.3f}%")

print(f"  {'α_G (×10⁻⁴⁵)':<20s} {alpha_G_tree * 1e45:14.3f} {alpha_G_corr * 1e45:14.3f}"
      f" {alpha_G_measured * 1e45:14.3f} {aG_tree_err:8.2f}% {aG_corr_err:8.3f}%")

print("  " + "-" * 66)
print(f"\n  Improvement factor: ~{G_tree_err / G_corr_err:.0f}×")

# Verify the correction is exactly Q²/(2Q-1)
ratio = alpha_G_corr / alpha_G_tree
expected_ratio = Q ** 2 / (2 * Q - 1)
print(f"\n  Correction ratio α_G(corr)/α_G(tree) = {ratio:.6f}")
print(f"  Q²/(2Q−1)                            = {expected_ratio:.6f}")
print(f"  Match: {'✓' if abs(ratio - expected_ratio) < 1e-10 else '✗'}")

# ============================================================
# JACOBSON'S η
# ============================================================

print(f"""
  {"=" * 66}
  JACOBSON'S ENTROPY DENSITY η
  {"=" * 66}

  Jacobson (1995) derived Einstein's equation as an equation of state,
  with Newton's constant set by an entropy density η that he left
  undetermined. The present framework provides:

    α_G = π⁴ Q² / [(ρQ)²²⁴ (2Q − 1)]

  Every factor is determined by the polynomial family and spacetime
  dimension:
    π⁴ = (π²)²    — squared projective volume (mass ratio → coupling)
    224 = 15 + 209 — electromagnetic + hierarchy staircase steps
    Q²/(2Q−1)      — gravitational screening (Ehrenfest correction)

  The resulting value of G:
    G_predicted = {G_corr:.5e} m³ kg⁻¹ s⁻²
    G_measured  = {G_measured:.5e} m³ kg⁻¹ s⁻²
    Error:        {G_corr_err:.3f}%
""")

# ============================================================
# THE −23 DISCRIMINANT BRIDGE
# ============================================================

print(f"  {'=' * 66}")
print(f"  THE −23 DISCRIMINANT BRIDGE")
print(f"  {'=' * 66}")
print(f"""
  Two independent computations in two different number fields:

  1. Norm of (2Q − 1) in ℚ(Q):
     The minimal polynomial of Q is f(x) = x⁴ − x − 1.
     N(2Q − 1) = 2⁴ × f(1/2) = 16 × [(1/2)⁴ − 1/2 − 1]
               = 16 × [1/16 − 3/2] = 16 × (−23/16) = −23""")

f_half = (0.5) ** 4 - 0.5 - 1
norm_2Q1 = 16 * f_half
print(f"\n     Computed: N(2Q − 1) = {norm_2Q1:.0f}")

# Discriminant of x³ − x − 1
disc = -4 * (-1) ** 3 - 27 * (-1) ** 2
print(f"""
  2. Discriminant of x³ − x − 1 (the polynomial defining ρ):
     Δ = −4(−1)³ − 27(−1)² = 4 − 27 = −23

     Computed: Δ = {disc}

  Both equal −23. The gravitational correction and the 3D polynomial
  share the same ramification prime. This is the algebraic bridge
  through which 3D and 4D communicate in gravitational physics.
""")

# ============================================================
# THE DEFICIT
# ============================================================

phi = (1 + np.sqrt(5)) / 2

print(f"  {'=' * 66}")
print(f"  THE DEFICIT: φ vs ρQ")
print(f"  {'=' * 66}")
print(f"""
  φ  = {phi:.5f}   (golden ratio — exact coherence in 2D)
  ρQ = {w:.5f}   (closest approach in 3+1 dimensions)

  Deficit: φ − ρQ = {phi - w:.5f}  ({(phi - w) / phi * 100:.3f}%)

  Unit-norm identity: N(ρQ) = −1, proving the relationship
  between ρQ and φ is algebraically exact, not approximate.

  Newton's constant is this deficit — the algebraic cost
  of living in three dimensions.
""")

GRAVITY CORRECTION REPRODUCER
The Dimensional Origin of Newton's Constant (Alexander, 2026)

  Roots:
    ρ  = 1.324718
    Q  = 1.220744
    ρQ = 1.61714

  The gravitational correction:
    λ₄² = (1 − 1/Q)²     = 0.03270   (3.27%)
    (2Q−1)/Q²             = 0.96730   (96.73%)
    1 − λ₄²               = 0.96730
    Identity: (2Q−1)/Q² ≡ 1 − λ₄²  ✓  (difference: 0.0e+00)

  Physical meaning:
    λ₄² = 3.27% of the 4D sector screens against itself
    (Ehrenfest's degeneracy: same-exponent cancellation in 4D)
    96.73% survives to contribute to gravitational binding

  TABLE 4: Gravitational predictions
  Quantity                 Tree level      Corrected       Measured  Tree err  Corr err
  ------------------------------------------------------------------
  G (×10⁻¹¹)                    6.456          6.674          6.674     3.27%    0.003%
  M_P/m_e (×10²²)               2.429          2.389          2.389     1.68%    0.001%
  α_G (×10⁻⁴⁵)                  1.694          1.752  